In [1]:
import scipy as sp
import numpy as np
import pandas as pd

In [2]:
class BasicModel:
    def check_value_and_set(self, name, value, allowed):
        message = "Wrong {}: '{}'\n\nAllowed values: {}"
        
        if value not in allowed:
            raise ValueError(message.format(
                name,
                value,
                allowed
            ))
        
        setattr(self, name, value)
        
    def check_value_type_and_set(self, name, value, allowed):
        message = "Wrong {} type: '{}'\n\nAllowed types: {}"
        
        if not isinstance(value, allowed):
            raise TypeError(message.format(
                name,
                value,
                allowed
            ))
        
        setattr(self, name, value)

# Logistic Regression

In [2]:
# train_df = pd.read_csv(
#     '../lab01/nyc-taxi-trip-duration/cleaned_train.csv'
# )

In [35]:
class LogisticRegression(BasicModel):
    def __init__(
        self,
        penalty='l2',
        tol=1e-4,
        C=1.0,
        fit_intercept=True,
        max_iter=100
    ):
        super().check_value_and_set(
            'penalty',
            penalty,
            ['l1', 'l2', None]
        )
        
        super().check_value_type_and_set(
            'tol',
            tol,
            (int, float)
        )
        
        super().check_value_type_and_set(
            'C',
            C,
            (int, float)
        )
        
        super().check_value_type_and_set(
            'fit_intercept',
            fit_intercept,
            bool
        )
        
        super().check_value_type_and_set(
            'max_iter',
            max_iter,
            int
        )
    
    def __get_l1_penalty(self):
        def l1_penalty(w):
            return 1/self.C * np.abs(w)
        
        def der_l1_penalty(w):
            # ignoring zeros existence
            return 1/self.C * ((w > 0) * 1 + (w <= 0) * -1)
        
        return l1_penalty, der_l1_penalty
    
    def __get_l2_penalty(self):
        def l2_penalty(w):
            return 1/self.C * np.multiply(w, w)
        
        def der_l2_penalty(w):
            return 2/self.C * w
        
        return l2_penalty, der_l2_penalty
    
    def __get_None_penalty(self):
        return None, None
    
    def fit(self, X, y, debug=False):
        assert \
            len(X.shape) == 2, \
            "X should be 2D vector"
        assert \
            y.shape == (X.shape[0], 1), \
            "y should be 2D vector and should correspond to X"
        
        if isinstance(X, pd.DataFrame):
            X = X.to_numpy()
        
        if isinstance(y, (pd.DataFrame, pd.Series)):
            y = y.to_numpy()
        
        if self.fit_intercept:
            X = np.hstack((
                X, 
                np.ones(
                    (X.shape[0], 1)
                )
            ))
        
        args = [X, y]
        
        args.extend(
            getattr(
                self,
                '_LogisticRegression__get_' + str(self.penalty) + '_penalty'
            )()
        )
        
        self.w = np.ones((X.shape[1], 1))
#         self.w = np.random.rand(X.shape[1], 1)
        
        if debug:
            return args
        
        result = sp.optimize.minimize(
            self.__cost,
            self.w,
            args,
            'L-BFGS-B',
            self.__gradient,
            tol=self.tol,
            options={
                'maxiter': self.max_iter
            }
        )
        
        assert result.success, result.message
        
        self.w = result.x
    
    @staticmethod
    def __predict(X, w):
        def predict_real(x, w):
            return x @ w

        def sigmoid(z):
            return 1 / (1 + np.exp(-z))
        
        return sigmoid(predict_real(X, w))
    
    @staticmethod
    def __cost(w, args):
        X, y, penalty, _ = args
        
        predictions = LogisticRegression.__predict(X, w)
        
        m = X.shape[0]
        
        cost0 = -(1 - y).T @ np.log(1 - predictions)
        cost1 = -y.T @ np.log(predictions)
        
        penalty_part = penalty(w).sum() if penalty else 0
        
        final_cost = (cost0 + cost1).sum() / m + penalty_part
        
        return final_cost
    
    def predict(self, X):
        if self.fit_intercept:
            X = np.hstack((
                X, 
                np.ones(
                    (X.shape[0], 1)
                )
            ))
        return self.__predict(X, self.w)
    
    @staticmethod
    def __gradient(w, args):
        X, y, _, der_penalty = args
        w = w.reshape((-1, 1))
        
        predictions = LogisticRegression.__predict(X, w)
        
        penalty_part = der_penalty(w) if der_penalty else 0
        
        return X.T @ (predictions - y) + penalty_part

In [36]:
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score
import unittest

def dummy_dataset():
    X, y = make_classification(100, 20)
    y = y.reshape((100, 1))
    return X, y

def prepare(debug=True, penalty=None):
    X, y = dummy_dataset()

    lr = LogisticRegression(penalty=penalty)

    args = lr.fit(X, y, debug)
    
    return lr, X, y, args

class TestLogisticRegression(unittest.TestCase):
    def test_gradient(self):
        lr, X, y, args = prepare()
        
        self.assertEqual(
            lr._LogisticRegression__gradient(lr.w, args).shape,
            (21, 1)
        )
        
    def test_cost(self):
        lr, X, y, args = prepare()
        
        self.assertEqual(
            type(lr._LogisticRegression__cost(lr.w, args)),
            np.float64
        )
    
    def test_None(self):
        lr, X, y, args = prepare(False)
        score = roc_auc_score(y, lr.predict(X))
        print("Score: {}".format(score))
        
    def test_l1(self):
        lr, X, y, args = prepare(False, 'l1')
        score = roc_auc_score(y, lr.predict(X))
        print("Score: {}".format(score))
    
    def test_l2(self):
        lr, X, y, args = prepare(False, 'l2')
        score = roc_auc_score(y, lr.predict(X))
        print("Score: {}".format(score))

In [41]:
unittest.main(argv=['first-arg-is-ignored', '--verbose'], exit=False)

test_None (__main__.TestLogisticRegression) ... ok
test_cost (__main__.TestLogisticRegression) ... ok
test_gradient (__main__.TestLogisticRegression) ... ok
test_l1 (__main__.TestLogisticRegression) ... ok
test_l2 (__main__.TestLogisticRegression) ... 

Score: 0.953525641025641
Score: 0.996


FAIL

FAIL: test_l2 (__main__.TestLogisticRegression)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-36-30bff43b0f2f>", line 47, in test_l2
    lr, X, y, args = prepare(False, 'l2')
  File "<ipython-input-36-30bff43b0f2f>", line 15, in prepare
    args = lr.fit(X, y, debug)
  File "<ipython-input-35-d255132e90f9>", line 111, in fit
    assert result.success, result.message
AssertionError: b'ABNORMAL_TERMINATION_IN_LNSRCH'

----------------------------------------------------------------------
Ran 5 tests in 0.065s

FAILED (failures=1)


Sometimes this happens due to too large l2 normalization.

In [27]:
unittest.main(argv=['first-arg-is-ignored', '--verbose'], exit=False)

test_None (__main__.TestLogisticRegression) ... /home/a3nippo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:126: RuntimeWarning: divide by zero encountered in log
/home/a3nippo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:126: RuntimeWarning: invalid value encountered in matmul
/home/a3nippo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:114: RuntimeWarning: overflow encountered in exp
/home/a3nippo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:127: RuntimeWarning: divide by zero encountered in log
/home/a3nippo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in matmul
ok
test_cost (__main__.TestLogisticRegression) ... ok
test_gradient (__main__.TestLogisticRegression) ... ok
test_l1 (__main__.TestLogisticRegression) ... ok
test_l2 (__main__.TestLogisticRegression) ... 

Score: 1.0
Score: 0.9867947178871549
Score: 1.0


ok

----------------------------------------------------------------------
Ran 5 tests in 0.042s

OK


# DecisionTree

In [3]:
class SplitFunctions:
    @staticmethod
    def gini_impurity(obj, observation_indexes):        
        node_y = obj.y[observation_indexes, :]

        p_0 = (node_y == 0).sum() / node_y.shape[0]
        p_1 = (node_y == 1).sum() / node_y.shape[0]

        return 1 - p_0 * p_0 - p_1 * p_1
    
    @staticmethod
    def gini_index(
        obj,
        _,
        left_split_indexes, 
        right_split_indexes
    ):
        w_left = left_split_indexes.shape[0]
        w_right = right_split_indexes.shape[0]

        W = w_left + w_right

        gini_index = w_left/W * SplitFunctions.gini_impurity(
            obj,
            left_split_indexes
        )

        gini_index += w_right/W * SplitFunctions.gini_impurity(
            obj,
            right_split_indexes
        )

        return gini_index
    
    @staticmethod
    def entropy(obj, observation_indexes):
        node_y = obj.y[observation_indexes, :]

        p_0 = (node_y == 0).sum() / node_y.shape[0]
        p_1 = (node_y == 1).sum() / node_y.shape[0]

        result = 0

        if p_0 != 0:
            result -= p_0*np.log2(p_0)

        if p_1 != 0:
            result -= p_1*np.log2(p_1)

        return result

    @staticmethod
    def information_gain(
        obj,
        observation_indexes,
        left_split_indexes,
        right_split_indexes
    ):
        entropy_before = SplitFunctions.entropy(
            obj,
            observation_indexes
        )

        entropy_after = SplitFunctions.entropy(
            obj,
            left_split_indexes
        )

        entropy_after += SplitFunctions.entropy(
            obj,
            right_split_indexes
        )

        return entropy_before - entropy_after

    @staticmethod
    def gain_ratio(
        obj,
        observation_indexes,
        left_split_indexes,
        right_split_indexes
    ):
        split_info = 0

        w_left = left_split_indexes.shape[0]
        if w_left != 0:
            split_info += w_left * np.log2(w_left)

        w_right = right_split_indexes.shape[0]
        if w_right != 0:
            split_info += w_right * np.log2(w_right)

        information_gain = SplitFunctions.information_gain(
            obj,
            observation_indexes,
            left_split_indexes,
            right_split_indexes
        )

        return information_gain / split_info

    @staticmethod
    def __mse(
        obj,
        observation_indexes
    ):
        labels = obj.y[observation_indexes, :]

        return ((labels - labels.mean())**2).mean()

    @staticmethod
    def mse_sum(
        obj,
        _,
        left_split_indexes,
        right_split_indexes
    ):
        left_mse = SplitFunctions.__mse(obj, left_split_indexes)

        right_mse = SplitFunctions.__mse(obj, right_split_indexes)

        return left_mse + right_mse

    @staticmethod
    def __mae(
        obj,
        observation_indexes
    ):
        labels = obj.y[observation_indexes, :]

        return (np.abs(labels - labels.mean())).sum()

    @staticmethod
    def mae_sum(
        obj,
        _,
        left_split_indexes,
        right_split_indexes
    ):
        left_mae = SplitFunctions.__mae(obj, left_split_indexes)

        right_mae = SplitFunctions.__mae(obj, right_split_indexes)

        return left_mae + right_mae

In [4]:
from collections import namedtuple
import functools

# CART DT (maybe)
class DecisionTree(BasicModel):
    class Node:
        def __init__(
            self,
            feature,
            value,
            observation_indexes,
            left=None,
            right=None,
            answer=None
        ):
            self.feature = feature
            self.split_value = value
            self.observation_indexes = observation_indexes
            self.left = left
            self.right = right
            self.answer = answer
            
        def is_leaf(self):
            return not self.left and not self.right
    
    criterion_name_to_calculator_method_name = {
        'gini': SplitFunctions.gini_index,
        'entropy': SplitFunctions.information_gain,
        'gain_ratio': SplitFunctions.gain_ratio,
        'mse': SplitFunctions.mse_sum,
        'mae': SplitFunctions.mae_sum,
    }
    
    criterion_name_to_cmp = {
        'gini': lambda x, y: x-y,
        'entropy': lambda x, y: y-x,
        'gain_ratio': lambda x, y: y-x,
        'mse': lambda x, y: y-x,
        'mae': lambda x, y: y-x,
    }
    
    def __init__(
        self,
        criterion='gini',
        splitter='best',
        max_depth=None,
        min_samples_split=2,
        max_features=None,
        random_state=42,
        debug=False
    ):
        super().check_value_and_set(
            'criterion',
            criterion,
            ['gini', 'entropy', 'gain_ratio', 'mse', 'mae']
        )
        
        self.__cmp_criterion_values = \
            self.criterion_name_to_cmp[criterion]
        self.__calc_criterion_value = \
            self.criterion_name_to_calculator_method_name[criterion]
        
        if criterion in ['mse', 'mae']:
            self.__task = 'regression'
        else:
            self.__task = 'classification'
        
        super().check_value_and_set(
            'splitter',
            splitter,
            ['best', 'random']
        )
        
        super().check_value_type_and_set(
            'max_depth',
            max_depth,
            (int, type(None))
        )
        if self.max_depth is None:
            self.max_depth = float('inf')
        
        super().check_value_type_and_set(
            'min_samples_split',
            min_samples_split,
            (int, float)
        )
        
        super().check_value_type_and_set(
            'max_features',
            max_features,
            (int, float, str, type(None))
        )
        
        if type(max_features) == str:
            super().check_value_and_set(
                'max_features',
                max_features,
                ['auto', 'sqrt', 'log2']
            )
        if max_features == 'auto':
            max_features = 'sqrt'
            
        super().check_value_type_and_set(
            'random_state',
            random_state,
            (np.random.RandomState, int)
        )
        if type(random_state) == int:
            self.random_state = np.random.RandomState(random_state)
            
        super().check_value_type_and_set(
            'debug',
            debug,
            bool
        )
        
        self.root = None
    
    def fit(self, X, y):
        assert \
            len(X.shape) == 2, \
            "X should be 2D vector"
        assert \
            y.shape == (X.shape[0], 1), \
            "y should be 2D vector and should correspond to X"
        
        if isinstance(X, pd.DataFrame):
            X = X.to_numpy()
        
        if isinstance(y, (pd.DataFrame, pd.Series)):
            y = y.to_numpy()
        
        self.X, self.y = X, y
        
        if self.max_features is None:
            self.max_features = float('inf')
        
        elif type(self.max_features) == float:
            self.max_features = np.floor(
                X.shape[1] * self.max_features
            )
        
        elif type(self.max_features) == str:
            self.max_features = np.floor(
                getattr(np, self.max_features)(X.shape[1])
            )
        
        if not self.debug:
            self.__construct_tree()
        
    def predict(self, X):
        assert self.root != None, "Not fitted"
        
        predictions = []
        
        for el in X:
            prediction = self.__predict_observation(el)
            predictions.append(prediction)
        
        return np.array(predictions).reshape((-1, 1))
            
    def __predict_observation(self, el):        
        node = self.root
        
        while not node.is_leaf():
            if el[node.feature] <= node.split_value:
                node = node.left
            else:
                node = node.right
        
        return node.answer
    
    def __get_split_pairs_gen(
        self,
        feature,
        observation_indexes
    ):
        node_x = self.X[observation_indexes, [feature]]
        
        observation_indexes, node_x = zip(
            *sorted(
                zip(
                    observation_indexes,
                    node_x.ravel()
                ), 
                key=lambda x: x[1]
            )
        )
        
        # to use numpy views
        observation_indexes = np.array(observation_indexes)
        
        uniques = list(dict.fromkeys(node_x).keys())
        
        last_i = 0
        id_unique = 0
        
        while id_unique < len(uniques) - 1:
            for i in range(last_i, len(node_x)):
                if node_x[i] > uniques[id_unique]:
                    last_i = i
                    break
            
            left_split_indexes = observation_indexes[:last_i]
            right_split_indexes = observation_indexes[last_i:]
            
            yield left_split_indexes, right_split_indexes, uniques[id_unique]
            
            id_unique += 1
    
    BestFeatureSplit = namedtuple(
        'BestFeatureSplit', 
        [
            'criterion_value',
            'split_value',
            'left_split_indexes',
            'right_split_indexes'
        ]
    )
    
    # output can be (None, None)
    def __find_best_split_by_feature(
        self, 
        feature, 
        observation_indexes
    ):
        split_pairs_gen = self.__get_split_pairs_gen(
            feature,
            observation_indexes
        )
        
        # find best split
        best_split = DecisionTree.BestFeatureSplit(
            None,
            None,
            None,
            None
        )
        
        for left_split_indexes, right_split_indexes, split_value in split_pairs_gen:
            criterion_value = self.__calc_criterion_value(
                self,
                observation_indexes,
                left_split_indexes,
                right_split_indexes
            )
            
            current_split = DecisionTree.BestFeatureSplit(
                criterion_value,
                split_value,
                left_split_indexes,
                right_split_indexes
            )
            
            if best_split.criterion_value is None or \
               self.__cmp_criterion_values(
                   criterion_value, 
                   best_split.criterion_value
               ) < 0:
                best_split = current_split
        
        return best_split

    def __construct_tree_helper(self, observation_indexes, depth):
        not_splitted_node = DecisionTree.Node(
            None,
            None,
            observation_indexes,
            answer=self.__get_answer(
                observation_indexes
            )
        )
        
        if depth > self.max_depth or \
           observation_indexes.shape[0] < self.min_samples_split:
            return not_splitted_node
        
        features = np.arange(self.X.shape[1])
        if self.splitter == 'random':
            self.random_state.shuffle(features)
        
        best_feature_split = None
        best_feature = None
    
        feature_counter = 0 if self.max_features else float('inf')
        
        for feature in features:
            if best_feature and \
               self.splitter == 'random' and \
               feature_counter > self.max_features:
                break
            
            feature_split = self.__find_best_split_by_feature(
                feature,
                observation_indexes
            )
            
            if feature_split.criterion_value is None:
                continue
            
            if best_feature is None or \
               self.__cmp_criterion_values(
                   feature_split.criterion_value,
                   best_feature_split.criterion_value
               ) < 0:
                best_feature_split = feature_split
                best_feature = feature
            
            feature_counter += 1
        
        if best_feature is None or \
           best_feature_split.criterion_value == 0:
            return not_splitted_node
        
        node = DecisionTree.Node(
            best_feature,
            best_feature_split.split_value,
            observation_indexes
        )
        
        node.left = self.__construct_tree_helper(
            best_feature_split.left_split_indexes,
            depth + 1
        )
        
        node.right = self.__construct_tree_helper(
            best_feature_split.right_split_indexes,
            depth + 1
        )
        
        if node.is_leaf():
            node.answer = self.__get_answer(
                observation_indexes
            )
        
        return node
    
    def __get_answer(self, observation_indexes):
        labels = self.y[observation_indexes, :]
        
        if self.__task == 'classification':
            ones = labels.sum()
            threshold = labels.shape[0] / 2
            
            if ones > threshold:
                return 1
            elif ones < threshold:
                return 0
            else:
                return self.random_state.randint(0, 1)
        else:
            return np.mean(labels)
    
    def __construct_tree(self):
        observation_indexes = np.arange(self.y.shape[0])
        
        self.root = self.__construct_tree_helper(
            observation_indexes, 
            1
        )
        
        if self.root is None:
            self.root = DecisionTree.Node(
                None,
                None,
                observation_indexes
            )
            
            self.answer = self.__get_answer(
                observation_indexes
            )

In [5]:
from sklearn.datasets import make_classification, make_regression
from sklearn.metrics import roc_auc_score, mean_squared_error, mean_absolute_error
import unittest
import time

cl_X, cl_y = make_classification(100, 20)
cl_y = cl_y.reshape((100, 1))

regr_X, regr_y = make_regression(100, 20)
regr_y = regr_y.reshape((100, 1))

def time_fit_predict(
    X, 
    y,
    score_names=['ROC AUC'],
    score_funcs=[roc_auc_score],
    *args,
    **kwargs
):
    start = time.time()
    
    dt = DecisionTree(*args, **kwargs)
    dt.fit(X, y)
    
    for score_name, score_func in zip(score_names, score_funcs):
        score = score_func(cl_y, dt.predict(cl_X))

        print("{} criterion {} score: {}".format(
            kwargs['criterion'].capitalize(), 
            score_name,
            score
        ))
    print("Time: {}".format(time.time() - start))

class TestDecisionTree(unittest.TestCase):
    def test_gini_impurity(self):
        dt = DecisionTree(debug=True)
        
        dt.fit(
            np.array([[1, 2], [1, 2], [1, 2]]),
            np.array([1, 0, 1]).reshape((-1, 1))
        )
        
        self.assertEqual(
            SplitFunctions.gini_impurity(
                dt,
                np.array([0, 2])
            ),
            0
        )
        
        self.assertEqual(
            SplitFunctions.gini_impurity(
                dt,
                np.array([0, 1])
            ),
            0.5
        )
    
    def test_gini(self):
        time_fit_predict(cl_X, cl_y, criterion='gini')
        
    def test_entropy(self):
        time_fit_predict(cl_X, cl_y, criterion='entropy')
    
    def test_gain_ratio(self):
        time_fit_predict(cl_X, cl_y, criterion='gain_ratio')
        
    def test_mse(self):
        time_fit_predict(
            regr_X, 
            regr_y,
            ['MSE', 'MAE'],
            [mean_squared_error, mean_absolute_error],
            criterion='mse'
        )
    
    def test_mae(self):
        time_fit_predict(
            regr_X, 
            regr_y,
            ['MSE', 'MAE'],
            [mean_squared_error, mean_absolute_error],
            criterion='mae'
        )
    
    def test_max_features_and_random_state(self):
        print('\nDifference in results means it works')
        
        for i in range(3):
            time_fit_predict(
                cl_X,
                cl_y,
                splitter='random',
                max_features='sqrt',
                criterion='gini',
                random_state=i
            )
    
    def test_max_depth(self):
        max_depth = 5
        
        def check_depth(node, depth=1):
            if not node.is_leaf():
                return check_depth(node.left, depth+1) and \
                       check_depth(node.right, depth+1)
            
            if depth > max_depth:
                result = False
            
            return True
        
        dt = DecisionTree(max_depth=max_depth)
        dt.fit(cl_X, cl_y)
        
        self.assertEqual(
            check_depth(dt.root),
            True
        )
    
    def test_min_samples_split(self):
        min_samples_split = 10
        
        def check_samples_split(node):
            if node.is_leaf():
                return True
            
            is_satisfied = \
                node.observation_indexes.shape[0] >= min_samples_split
            
            return is_satisfied and \
                   check_samples_split(node.left) and \
                   check_samples_split(node.right)
        
        dt = DecisionTree(min_samples_split=min_samples_split)
        dt.fit(cl_X, cl_y)
        
        self.assertEqual(
            check_samples_split(dt.root),
            True
        )

In [6]:
unittest.main(argv=['first-arg-is-ignored', '--verbose'], exit=False)

test_entropy (__main__.TestDecisionTree) ... ok
test_gain_ratio (__main__.TestDecisionTree) ... 

Entropy criterion ROC AUC score: 1.0
Time: 1.8193128108978271


ok
test_gini (__main__.TestDecisionTree) ... 

Gain_ratio criterion ROC AUC score: 1.0
Time: 1.956484079360962


ok
test_gini_impurity (__main__.TestDecisionTree) ... ok
test_mae (__main__.TestDecisionTree) ... 

...

ok
test_min_samples_split (__main__.TestDecisionTree) ... 

Gini criterion ROC AUC score: 0.94
Time: 0.1279444694519043


ok
test_mse (__main__.TestDecisionTree) ... 

Mse criterion MSE score: 3199.892861356935
Mse criterion MAE score: 42.80670221302634
Time: 1.31022047996521


ok

----------------------------------------------------------------------
Ran 9 tests in 6.846s

OK


entropy and gain_ratio are slower due to calculation of logarithm.